# Pandas Examples

Batfish questions can return a huge amount of data, which you may want to filter in various ways based on your task. While most Batfish questions support basic filtering, they may not support your desired filtering criteria. Further, for performance, you may want to fetch the answer once and filter it using multiple different criteria. These scenarios are where Pandas-based filtering can help. 

Batfish answers can be easily turned into a [Pandas Dataframe](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.html) (using `.frame()`), after which you can use the full power of Pandas to filter and manipulate data. This notebook provides a few examples of common manipulations for Batfish. It is not intended as a complete guide of Pandas data manipulation.

![Analytics](https://ga-beacon.appspot.com/UA-100596389-3/open-source/pybatfish/jupyter_notebooks/pandas-examples?pixel&useReferer) 

Let's first initialize a snapshot that we will use in our examples.

In [1]:
# Import packages
%run startup.py
bf = Session(host="localhost")

# Initialize a network and a snapshot
bf.set_network("pandas-example")

SNAPSHOT_NAME = "snapshot"
SNAPSHOT_PATH = "networks/hybrid-cloud/"
bf.init_snapshot(SNAPSHOT_PATH, name=SNAPSHOT_NAME, overwrite=True)

Your snapshot was successfully initialized but Batfish failed to fully recognized some lines in one or more input files. Some unrecognized configuration lines are not uncommon for new networks, and it is often fine to proceed with further analysis. You can help the Batfish developers improve support for your network by running:

    bf.upload_diagnostics(dry_run=False, contact_info='<optional email address>')

to share private, anonymized information. For more information, see the documentation with:

    help(bf.upload_diagnostics)


'snapshot'

## Filtering `initIssues`

After initializing the snapshot, you often want to look at the initIssues answer. If there are too many issues, you may want to ignore a particular class of issues. We show below how to do that. 

In [2]:
# Lets get the initIssues for our snapshot
issues = bf.q.initIssues().answer().frame()
issues

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
0,['leaf1'],None,Convert warning (redflag),Interface Ethernet12 has an undefined channel group Port-Channel20,None,None
1,None,"[configs/Leaf2.cfg:[6], configs/Leaf4.cfg:[6], configs/Leaf1.cfg:[6], configs/Leaf3.cfg:[6], configs/Spine1.cfg:[6], configs/Spine2.cfg:[6]]",Parse warning,This syntax is unrecognized,transceiver qsfp default-mode 4x10G,[arista_configuration]
2,None,[aws_configs:[]],Parse warning (unimplemented),Unrecognized element 'ServiceDetails' in AWS file aws_configs/us-west-2/VpcEndpointServices.json,None,None
3,None,[aws_configs:[]],Parse warning (unimplemented),Unrecognized element 'ServiceDetails' in AWS file aws_configs/us-east-2/VpcEndpointServices.json,None,None


In [3]:
# Ignore all issues whose Line_Text contain one of these as a substring
ignore_line_texts = [
    "transceiver"
]

issues[issues.apply(
                lambda row: all([row['Line_Text'] is None or 
                                 line_text not in row['Line_Text'] for line_text in ignore_line_texts]), 
                axis=1)]

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
0,['leaf1'],None,Convert warning (redflag),Interface Ethernet12 has an undefined channel group Port-Channel20,None,None
2,None,[aws_configs:[]],Parse warning (unimplemented),Unrecognized element 'ServiceDetails' in AWS file aws_configs/us-west-2/VpcEndpointServices.json,None,None
3,None,[aws_configs:[]],Parse warning (unimplemented),Unrecognized element 'ServiceDetails' in AWS file aws_configs/us-east-2/VpcEndpointServices.json,None,None


In the code above, we are using [Pandas's apply method](https://pandas.pydata.org/docs/reference/api/pandas.DataFrame.apply.html) to map rows in the `issues` DataFrame to a binary array. A row maps to True if its `Line_Text` is `None` or does not contain any of the substrings in the `ignoredDetails` list. The Python method [all()](https://docs.python.org/3/library/functions.html#all) returns `True` if all elements of the input iterable are `True`. Using the binary array as a filter for `issues` produces rows that match our criterion. 

Instead of ignoring some issues, you may want to look only at issues that match a certain criteria. That too can be easily accomplished, as follows.

In [4]:
# Only show issues whose details match these substrings
focus_details = [
    "Unrecognized element 'ServiceDetails' in AWS"
]

issues[issues.apply(
                lambda row: any([detail in row['Details'] for detail in focus_details]), 
                axis=1)]

,Nodes,Source_Lines,Type,Details,Line_Text,Parser_Context
2,None,[aws_configs:[]],Parse warning (unimplemented),Unrecognized element 'ServiceDetails' in AWS file aws_configs/us-west-2/VpcEndpointServices.json,None,None
3,None,[aws_configs:[]],Parse warning (unimplemented),Unrecognized element 'ServiceDetails' in AWS file aws_configs/us-east-2/VpcEndpointServices.json,None,None


The code above is similar to the one we used earlier. The main difference is the use of Python method [any()](https://docs.python.org/3/library/functions.html#any) that returns `True` if any element of the input iterable is `True`.

## Filtering objects

In [5]:
# Fetch interface properties and display its first five rows
interfaces = bf.q.interfaceProperties().answer().frame()
interfaces.head(5)

,Interface,Access_VLAN,Active,Admin_Up,All_Prefixes,Allowed_VLANs,Auto_State_VLAN,Bandwidth,Blacklisted,Channel_Group,Channel_Group_Members,DHCP_Relay_Addresses,Declared_Names,Description,Encapsulation_VLAN,HSRP_Groups,HSRP_Version,Inactive_Reason,Incoming_Filter_Name,MLAG_ID,MTU,Native_VLAN,Outgoing_Filter_Name,PBR_Policy_Name,Primary_Address,Primary_Network,Proxy_ARP,Rip_Enabled,Rip_Passive,Spanning_Tree_Portfast,Speed,Switchport,Switchport_Mode,Switchport_Trunk_Encapsulation,VRF,VRRP_Groups,Zone_Name
0,__aws-services-gateway__[aws-services],None,True,True,[],,True,1e+12,False,None,[],[],[],To AWS services,None,[],None,,None,None,1500,None,None,None,link-local:169.254.0.1,None,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
1,__aws-services-gateway__[backbone],None,True,True,[],,True,1e+12,False,None,[],[],[],To AWS backbone,None,[],None,,None,None,1500,None,None,None,link-local:169.254.0.1,None,False,False,False,False,None,False,NONE,DOT1Q,default,[],None
2,exitgw[GigabitEthernet1],None,True,True,['10.10.100.2/24'],,True,1e+09,False,None,[],[],['GigabitEthernet1'],None,None,[],None,,None,None,1500,None,None,None,10.10.100.2/24,10.10.100.0/24,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
3,exitgw[GigabitEthernet2],None,True,True,['10.10.101.2/24'],,True,1e+09,False,None,[],[],['GigabitEthernet2'],None,None,[],None,,None,None,1500,None,None,None,10.10.101.2/24,10.10.101.0/24,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
4,exitgw[GigabitEthernet3],None,True,True,['147.75.69.27/31'],,True,1e+09,False,None,[],[],['GigabitEthernet3'],None,None,[],None,,None,None,1500,None,None,None,147.75.69.27/31,147.75.69.26/31,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None


To filter based on a column, we need to know its data type. We can learn that in the [batfish documentation](https://pybatfish.readthedocs.io/en/latest/questions.html) or using Python's `type()` method:

In [6]:
print("Type of 'Interface' column is {}".format(type(interfaces['Interface'][0])))
print("Type of 'Active' column is {}".format(type(interfaces['Active'][0])))

Type of 'Interface' column is <class 'pybatfish.datamodel.primitives.Interface'>
Type of 'Active' column is <class 'bool'>


We show three examples of filterining based on these two columns. They use the fact that the class [`pybatfish.datamodel.primitives.Interface`](https://pybatfish.readthedocs.io/en/latest/datamodel.html#pybatfish.datamodel.primitives.Interface) has `hostname` and `interface` properties (both of which are strings). 

In [7]:
# Display all interfaces on node 'exitgw'
interfaces[interfaces.apply(
                lambda row: row['Interface'].hostname == 'exitgw', 
                axis=1)]

,Interface,Access_VLAN,Active,Admin_Up,All_Prefixes,Allowed_VLANs,Auto_State_VLAN,Bandwidth,Blacklisted,Channel_Group,Channel_Group_Members,DHCP_Relay_Addresses,Declared_Names,Description,Encapsulation_VLAN,HSRP_Groups,HSRP_Version,Inactive_Reason,Incoming_Filter_Name,MLAG_ID,MTU,Native_VLAN,Outgoing_Filter_Name,PBR_Policy_Name,Primary_Address,Primary_Network,Proxy_ARP,Rip_Enabled,Rip_Passive,Spanning_Tree_Portfast,Speed,Switchport,Switchport_Mode,Switchport_Trunk_Encapsulation,VRF,VRRP_Groups,Zone_Name
2,exitgw[GigabitEthernet1],None,True,True,['10.10.100.2/24'],,True,1e+09,False,None,[],[],['GigabitEthernet1'],None,None,[],None,,None,None,1500,None,None,None,10.10.100.2/24,10.10.100.0/24,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
3,exitgw[GigabitEthernet2],None,True,True,['10.10.101.2/24'],,True,1e+09,False,None,[],[],['GigabitEthernet2'],None,None,[],None,,None,None,1500,None,None,None,10.10.101.2/24,10.10.101.0/24,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
4,exitgw[GigabitEthernet3],None,True,True,['147.75.69.27/31'],,True,1e+09,False,None,[],[],['GigabitEthernet3'],None,None,[],None,,None,None,1500,None,None,None,147.75.69.27/31,147.75.69.26/31,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
5,exitgw[GigabitEthernet4],None,False,False,[],,True,1e+09,False,None,[],[],['GigabitEthernet4'],None,None,[],None,Administratively down,None,None,1500,None,None,None,None,None,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
6,exitgw[Loopback0],None,True,True,['2.2.2.2/32'],,True,8e+09,None,None,[],[],['Loopback0'],None,None,[],None,,None,None,1500,None,None,None,2.2.2.2/32,2.2.2.2/32,True,False,False,False,None,False,NONE,DOT1Q,default,[],None
7,exitgw[Loopback123],None,True,True,['192.168.123.7/32'],,True,8e+09,None,None,[],[],['Loopback123'],None,None,[],None,,None,None,1500,None,None,None,192.168.123.7/32,192.168.123.7/32,True,False,False,False,None,False,NONE,DOT1Q,default,[],None
8,exitgw[Tunnel1],None,True,True,['169.254.25.162/30'],,True,100000,None,None,[],[],['Tunnel1'],None,None,[],None,,None,None,1500,None,None,None,169.254.25.162/30,169.254.25.160/30,True,False,False,False,None,False,NONE,DOT1Q,default,[],None
9,exitgw[Tunnel2],None,True,True,['169.254.172.2/30'],,True,100000,None,None,[],[],['Tunnel2'],None,None,[],None,,None,None,1500,None,None,None,169.254.172.2/30,169.254.172.0/30,True,False,False,False,None,False,NONE,DOT1Q,default,[],None
10,exitgw[Tunnel3],None,True,True,['169.254.252.78/30'],,True,100000,None,None,[],[],['Tunnel3'],None,None,[],None,,None,None,1500,None,None,None,169.254.252.78/30,169.254.252.76/30,True,False,False,False,None,False,NONE,DOT1Q,default,[],None
11,exitgw[Tunnel4],None,True,True,['169.254.215.82/30'],,True,100000,None,None,[],[],['Tunnel4'],None,None,[],None,,None,None,1500,None,None,None,169.254.215.82/30,169.254.215.80/30,True,False,False,False,None,False,NONE,DOT1Q,default,[],None


In [8]:
# Display all GigabitEthernet interfaces on node 'exitgw'
interfaces[interfaces.apply(
                lambda row: row['Interface'].hostname == 'exitgw' and 
                            row['Interface'].interface.startswith('GigabitEthernet'), 
                axis=1)]

,Interface,Access_VLAN,Active,Admin_Up,All_Prefixes,Allowed_VLANs,Auto_State_VLAN,Bandwidth,Blacklisted,Channel_Group,Channel_Group_Members,DHCP_Relay_Addresses,Declared_Names,Description,Encapsulation_VLAN,HSRP_Groups,HSRP_Version,Inactive_Reason,Incoming_Filter_Name,MLAG_ID,MTU,Native_VLAN,Outgoing_Filter_Name,PBR_Policy_Name,Primary_Address,Primary_Network,Proxy_ARP,Rip_Enabled,Rip_Passive,Spanning_Tree_Portfast,Speed,Switchport,Switchport_Mode,Switchport_Trunk_Encapsulation,VRF,VRRP_Groups,Zone_Name
2,exitgw[GigabitEthernet1],None,True,True,['10.10.100.2/24'],,True,1e+09,False,None,[],[],['GigabitEthernet1'],None,None,[],None,,None,None,1500,None,None,None,10.10.100.2/24,10.10.100.0/24,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
3,exitgw[GigabitEthernet2],None,True,True,['10.10.101.2/24'],,True,1e+09,False,None,[],[],['GigabitEthernet2'],None,None,[],None,,None,None,1500,None,None,None,10.10.101.2/24,10.10.101.0/24,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
4,exitgw[GigabitEthernet3],None,True,True,['147.75.69.27/31'],,True,1e+09,False,None,[],[],['GigabitEthernet3'],None,None,[],None,,None,None,1500,None,None,None,147.75.69.27/31,147.75.69.26/31,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
5,exitgw[GigabitEthernet4],None,False,False,[],,True,1e+09,False,None,[],[],['GigabitEthernet4'],None,None,[],None,Administratively down,None,None,1500,None,None,None,None,None,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None


In [9]:
# Display all active GigabitEthernet interfaces on node 'exitgw'
interfaces[interfaces.apply(
                lambda row: row['Interface'].hostname == 'exitgw' and 
                            row['Interface'].interface.startswith('GigabitEthernet') and 
                            row['Active'], 
                axis=1)]

,Interface,Access_VLAN,Active,Admin_Up,All_Prefixes,Allowed_VLANs,Auto_State_VLAN,Bandwidth,Blacklisted,Channel_Group,Channel_Group_Members,DHCP_Relay_Addresses,Declared_Names,Description,Encapsulation_VLAN,HSRP_Groups,HSRP_Version,Inactive_Reason,Incoming_Filter_Name,MLAG_ID,MTU,Native_VLAN,Outgoing_Filter_Name,PBR_Policy_Name,Primary_Address,Primary_Network,Proxy_ARP,Rip_Enabled,Rip_Passive,Spanning_Tree_Portfast,Speed,Switchport,Switchport_Mode,Switchport_Trunk_Encapsulation,VRF,VRRP_Groups,Zone_Name
2,exitgw[GigabitEthernet1],None,True,True,['10.10.100.2/24'],,True,1e+09,False,None,[],[],['GigabitEthernet1'],None,None,[],None,,None,None,1500,None,None,None,10.10.100.2/24,10.10.100.0/24,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
3,exitgw[GigabitEthernet2],None,True,True,['10.10.101.2/24'],,True,1e+09,False,None,[],[],['GigabitEthernet2'],None,None,[],None,,None,None,1500,None,None,None,10.10.101.2/24,10.10.101.0/24,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None
4,exitgw[GigabitEthernet3],None,True,True,['147.75.69.27/31'],,True,1e+09,False,None,[],[],['GigabitEthernet3'],None,None,[],None,,None,None,1500,None,None,None,147.75.69.27/31,147.75.69.26/31,True,False,False,False,1e+09,False,NONE,DOT1Q,default,[],None


## Filtering columns

When viewing Batfish answers, you may want to view only some of the columns. Pandas makes that easy for both original answers and answers where some rows have been filtered, as both of them are just DataFrames.

In [10]:
# Filter interfaces to all active GigabitEthernet interfaces on node exitgw
exitgw_gige_active_interfaces = interfaces[interfaces.apply(
                        lambda row: row['Interface'].hostname == 'exitgw' and 
                                    row['Interface'].interface.startswith('GigabitEthernet') and 
                                    row['Active'], 
                axis=1)]
# Display only the Interface and All_Prefixes columns of the filtered DataFrame
exitgw_gige_active_interfaces[["Interface", "All_Prefixes"]]

,Interface,All_Prefixes
2,exitgw[GigabitEthernet1],['10.10.100.2/24']
3,exitgw[GigabitEthernet2],['10.10.101.2/24']
4,exitgw[GigabitEthernet3],['147.75.69.27/31']


## Counting rows

Often, you would be interested in counting the number of rows in the filtered answer. This is super easy because Python's `len()` method, which we use for iterables, can be used on DataFrames as well.

In [11]:
# Show the number of rows in the filtered DataFrame that we obtained above
len(exitgw_gige_active_interfaces)

3

## Summary

In this notebook, we showed how you can use Pandas filter manipulate Batfish answers, including filtering rows to those that match the desired criteria and filtering columns.

***
### Get involved with the Batfish community

Join our community on [Slack](https://join.slack.com/t/batfish-org/shared_invite/enQtMzA0Nzg2OTAzNzQ1LTcyYzY3M2Q0NWUyYTRhYjdlM2IzYzRhZGU1NWFlNGU2MzlhNDY3OTJmMDIyMjQzYmRlNjhkMTRjNWIwNTUwNTQ) and [GitHub](https://github.com/batfish/batfish). 